In [ ]:
!mamba install t-coffee mmseqs2 -y -q

In [1]:
from learnMSA.msa_hmm.MsaHmmLayer import MsaHmmLayer
from learnMSA.msa_hmm.MsaHmmCell import MsaHmmCell
import numpy as np
import tensorflow as tf

class PermuteSeqs(tf.keras.layers.Layer):
    def __init__(self, perm, **kwargs):
        super(PermuteSeqs, self).__init__(**kwargs)
        self.perm = perm

    def call(self, sequences):
        return tf.transpose(sequences, self.perm, name="loglik")

    def get_config(self):
        return {"perm": self.perm}


sequences = tf.keras.Input(shape=(None,None,20), name="sequences", dtype=tf.float32)
hmm_cell = MsaHmmCell([10])
msa_hmm_layer = MsaHmmLayer(hmm_cell)
s = PermuteSeqs([1,0,2,3])(sequences)
map_loss, loglik = msa_hmm_layer(s)
model = tf.keras.Model(inputs=(sequences), outputs=(map_loss, loglik))
model.compile(optimizer=tf.optimizers.Adam(0.1), loss=lambda _,loss: loss) 

data = np.zeros((100, 1, 30, 20), dtype=np.float32)

model.fit(data, np.ones((100), dtype=np.float32), epochs=1, steps_per_epoch=20, batch_size=32)

2024-10-11 08:55:37.835678: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-11 08:55:37.852025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 08:55:37.869479: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 08:55:37.874800: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 08:55:37.889604: I tensorflow/core/platform/cpu_feature_guar

(35, 35)
(94, 2)
(94,)
(35, 35)
(94, 2)
(94,)
(35, 35)
(94, 2)
(94,)
(35, 35)
(94, 2)
(94,)
(35, 35)
(94, 2)
(94,)
(35, 35)
(94, 2)
(94,)
(35, 35)
(94, 2)
(94,)
(35, 35)
(94, 2)
(94,)


I0000 00:00:1728636941.581011   32367 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728636941.581254   32367 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728636941.581373   32367 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728636941.581534   32367 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

(35, 35)
(94, 2)
(94,)
(35, 35)
(94, 2)
(94,)
(35, 35)
(94, 2)
(94,)


I0000 00:00:1728636942.182012   32367 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728636942.182258   32367 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728636942.182378   32367 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728636942.182539   32367 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

...

I0000 00:00:1728636945.218882   32488 service.cc:146] XLA service 0x555edf3042a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728636945.218907   32488 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-11 08:55:45.436610: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-11 08:55:46.012608: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-10-11 08:55:46.321457: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:762] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility

 1/20 ━━━━━━━━━━━━━━━━━━━━ 2:25 8s/step - loss: -295734400.0000

I0000 00:00:1728636949.629254   32488 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 555ms/step - loss: -329612000.0000


2024-10-11 08:56:00.194540: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-11 08:56:00.195177: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
/home/beckerf/mambaforge/envs/learnMSAdev2/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


In [46]:
a = np.array([[0,2], [0,1], [0,0], [5,2], [3,2], [1,1], [5,0]])
x = np.argsort([i*6+j for i,j in a])
# -> [1,0,3,2]
a[x][np.argsort(x)]

array([[0, 2],
       [0, 1],
       [0, 0],
       [5, 2],
       [3, 2],
       [1, 1],
       [5, 0]])

In [28]:
import numpy as np
import tensorflow as tf
tf.config.optimizer.set_jit(False)

batch_size = 4
input_shape = (3, 3)

indices = np.array([[0, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 1], [2, 0, 0], [2, 0, 1], [3, 0, 0], [3, 0, 1]])
inputs = tf.sparse.SparseTensor(dense_shape=(batch_size, *input_shape),
                                indices=indices,
                                values=[9., 1, 9, 1, 9, 1, 9, 1])


@tf.function(input_signature=[tf.SparseTensorSpec(
    shape=(4, 3, 3),
    dtype=tf.dtypes.float32)], jit_compile=True)
def get_batch_sum(inputs):
    # same problem with tf.sparse.reduce_max
    return tf.sparse.reduce_sum(inputs)


sum_out = get_batch_sum(inputs)
print(sum_out)

I0000 00:00:1728635078.961448    8385 service.cc:146] XLA service 0x55e2e5052900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728635078.961466    8385 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-11 08:24:38.962095: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at xla_ops.cc:577 : INVALID_ARGUMENT: Detected unsupported operations when trying to compile graph __inference_get_batch_sum_416[_XlaMustCompile=true,config_proto=16962375805000911571,executor_type=11160318154034397263] on XLA_GPU_JIT: SparseReduceSum (No registered 'SparseReduceSum' OpKernel for XLA_GPU_JIT devices compatible with node {{node SparseReduceSum}}){{node SparseReduceSum}}
The op is created at: 
File "<frozen runpy>", line 198, in _run_module_as_main
File "<frozen runpy>", line 88, in _run_code
File "/home/beckerf/mambaforge/envs/learnMSAdev2/lib/python3.12/site-packages/ipykernel_launc

InvalidArgumentError: Detected unsupported operations when trying to compile graph __inference_get_batch_sum_416[_XlaMustCompile=true,config_proto=16962375805000911571,executor_type=11160318154034397263] on XLA_GPU_JIT: SparseReduceSum (No registered 'SparseReduceSum' OpKernel for XLA_GPU_JIT devices compatible with node {{node SparseReduceSum}}){{node SparseReduceSum}}
The op is created at: 
File "<frozen runpy>", line 198, in _run_module_as_main
File "<frozen runpy>", line 88, in _run_code
File "/home/beckerf/mambaforge/envs/learnMSAdev2/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
File "/home/beckerf/mambaforge/envs/learnMSAdev2/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
File "/home/beckerf/mambaforge/envs/learnMSAdev2/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
File "/home/beckerf/mambaforge/envs/learnMSAdev2/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205

In [13]:
st_a

SparseTensor(indices=tf.Tensor(
[[0 2]
 [3 4]], shape=(2, 2), dtype=int64), values=tf.Tensor([31.  2.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([ 4 10], shape=(2,), dtype=int64))

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from learnMSA.msa_hmm import Configuration, Align, Visualize
from learnMSA.msa_hmm.SequenceDataset import SequenceDataset

2024-10-11 09:01:23.583040: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-11 09:01:23.595036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 09:01:23.606656: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 09:01:23.610111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 09:01:23.620799: I tensorflow/core/platform/cpu_feature_guar

## learnMSA demo

In this notebook, we will fit a number of HMM models to a dataset of unaligned sequences. 
We will then use the fitted models to align the sequences and compare the results.
Moreover, we will visualize the best (according to an objective criterion) model and alignment.

*This notebook is meant to be a demo for running learnMSA in Python code. Check the readme if you want to run learnMSA from the command line.* 

Change the variables in the following cell to fit your needs.

In [2]:
# Your fasta file with unaligned sequences.

train_filename = "test/data/egf.fasta"

# Reference file with aligned sequences that have matching IDs to (potentially a subset of) the 
# sequences in the train_file.
# Replace with empty string if no reference is available.
ref_filename = "test/data/egf.ref"

# The number of independently trained models.
num_models = 4

# Use sequence weights based on a rapid pre-clustering of the sequences (requires mmseqs2 to be installed)
use_weights = True

# Align long insertions with an external aligner left unaligned by the main MSA stage (requires famsa to be installed).
align_insertions = True

## Run learnMSA from Python (Training + Viterbi alignment)

In [3]:
out_filename = "test/data/interactive.alignment.fasta"
config = Configuration.make_default(num_models)
with SequenceDataset(train_filename, fmt="fasta") as data:
    alignment_model = Align.run_learnMSA(data,
                                        out_filename,
                                        config, 
                                        sequence_weights=Align.compute_sequence_weights(train_filename, "tmp") if use_weights else None,
                                        verbose=True,
                                        align_insertions=align_insertions)
    Visualize.print_and_plot(alignment_model, alignment_model.best_model)

Training of 4 models on file egf.fasta
Configuration: 
{
num_models : 4
transitioner : ProfileHMMTransitioner(
 transition_init=
    {
    begin_to_match : DefaultEntry() , match_to_end : DefaultExit() , 
    match_to_match : DefaultMatchTransition(1) , match_to_insert : DefaultMatchTransition(-1) , 
    insert_to_match : Norm(0, 0.1) , insert_to_insert : Norm(-0.5, 0.1) , 
    match_to_delete : DefaultMatchTransition(-1) , delete_to_match : Norm(0, 0.1) , 
    delete_to_delete : Norm(-0.5, 0.1) , left_flank_loop : Norm(0, 0.1) , 
    left_flank_exit : Norm(-1, 0.1) , right_flank_loop : Norm(0, 0.1) , 
    right_flank_exit : Norm(-1, 0.1) , unannotated_segment_loop : Norm(0, 0.1) , 
    unannotated_segment_exit : Norm(-1, 0.1) , end_to_unannotated_segment : Norm(-9, 0.1) , 
    end_to_right_flank : Norm(0, 0.1) , end_to_terminal : Norm(0, 0.1)
    },
 flank_init=Const(0.0),
 prior=ProfileHMMTransitionPrior(match_comp=1, insert_comp=1, delete_comp=1, alpha_flank=7000, alpha_single=10000

I0000 00:00:1728637296.061508   42837 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728637296.061758   42837 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728637296.061881   42837 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728637296.062047   42837 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/10


I0000 00:00:1728637297.715024   42837 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728637297.715262   42837 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728637297.715381   42837 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728637297.715543   42837 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/functional_4_1/anc_probs_layer_1/matrix_exponential/while/functional_4_1/anc_probs_layer_1/matrix_exponential/while/Less_0/accumulator defined at (most recent call last):
<stack traces unavailable>
XLA compilation requires a fixed tensor list size. Set the max number of elements. This could also happen if you're using a TensorArray in a while loop that does not have its maximum_iteration set, you can fix this by setting maximum_iteration to a suitable value.

Stack trace for op definition: 
File "<frozen runpy>", line 198, in _run_module_as_main
File "<frozen runpy>", line 88, in _run_code
File "/home/beckerf/mambaforge/envs/learnMSAdev2/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
File "/home/beckerf/mambaforge/envs/learnMSAdev2/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
File "/home/beckerf/mambaforge/envs/learnMSAdev2/lib/python3.